<a href="https://colab.research.google.com/github/BioML-UGent/MLLS/blob/main/13_rnns/PClab_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNNs

## Introduction

Recurrent networks strictly operate on 1-D sequences. They can be used for a variety of tasks, pictured below:

<img src="http://karpathy.github.io/assets/rnn/diags.jpeg" width = 500>

Examples of the settings in the picture:
- one to one: vanilla MLPs that map a fixed size 1-D vector to a 1-D vector for classification or regression
- one to many: Image captioning, given an input embedding (obtained with a CNN), a textual caption of variable length is generated.
- many to one: (1) Sentence classification such as sentiment analysis or (2) image generation from text: in both cases variable input texts are given as input and a fixed dimensional output is generated.
- many to many: (1) machine translation of a variable-length sentence to another variable-length sentence or (2) transcription of a variable-length .mp3 audio to a variable length text.
- many to many (1to1 correspondence): (1) Video classification: one label for a variable number of frames in the video (the video frame embedding can be obtained with a CNN and then input into a RNN), (2) autoregressive language modeling: trying to predict the next word in the sentence, for generative purposes or (3) word classification: classify every word as belonging to a category.

Note that these settings are not exclusive to recurrent neural networks. In fact, any network type that works on variable input sequences can be used towards these ends. Most famously of which are of course, Transformers, which have all but replaced RNNs in NLP and many other fields. An explanation and implementation of transformers is out of the scope of this course. It suffices to know that RNNs process input sequence sequentially through memory cells, whereas transformers do it in parallel through an $n \times n$ attention matrix. Other than RNNs and Transformers, convolutional networks can also be used on variable length inputs: a 1D kernel can equally well convolve over a sequence of length $100$ as $1000$. It is only because of the linear layers at the end for classification requiring a specific number of input nodes that typical CNNs become applicable on only one specific input size.

## Partim 1: Autoregressive modeling

Autoregressive modeling is the task of trying to predict the next token in a sequence given which tokens came before it: $P(x_i | x_{i-1}, x_{i-2}, ..., x_1)$.

In this PC-lab we will explore autoregressive modeling on the beloved dataset of the first project: bicycle traffic prediction

<img src="https://images0.persgroep.net/rcs/4cQwm-ofvb3eyIKMWnNf5axxLHg/diocontent/217261403/_fitwidth/694/?appId=21791a8992982cd8da851550a453bd7f&quality=0.8" width = 500>

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/BioML-UGent/MLLS/main/13_rnns/train_data.csv", "./train_data.csv")
urllib.request.urlretrieve("https://raw.githubusercontent.com/BioML-UGent/MLLS/main/13_rnns/test_data.csv", "./test_data.csv")

train_data = pd.read_csv("train_data.csv", sep = ",")
test_data = pd.read_csv("test_data.csv", sep = ",")

train_data

Let's encode the Hour as separate feature

In [ ]:
train_data["Hour"] = train_data["Date_hour"].str.split("T", expand = True)[1].astype(float)
test_data["Hour"] = test_data["Date_hour"].str.split("T", expand = True)[1].astype(float)
train_data.head()

Architecturally, autoregressive modeling of a timeseries looks like this:

<img src="https://raw.githubusercontent.com/gdewael/teaching/main/predmod/RNNs/AR.png" width = 500>

For every point in the timeseries, the *input token* consists of the value at that timepoint and, optionally, extra covariates pertaining to that timepoint. Conceptually, this is similar to the channels in a CNN, as there we also had input tokens (e.g. pixels) with multiple values (e.g. the 3 RGB values) per token. In our case, extra covariates could be for example the hour at which that timepoint was taken. This input sequence will go into the RNN, which will keep a hidden layer which acts as a memory bank. The memory bank of every input will consist of a combination of the information at that time point and the information coming in from the memory cell at the previous time point. The specific way this information is brought together depends on the specific construction of the RNN. We refer you to the theory lectures for details. The most popular constructions are the LSTM and the GRU memory cells. For every timestep, the model outputs a vector which (for this purpose) needs to be linearly recombined to one number as our goal is to predict the value of the next timestep (i.e. a regression task).

Code-wise, it is important to know that for a given sequence, we have an input $x$ consisting of the timepoints in that sequence, and an output $y$, consisting of the same points, but **shifted one time-step to the left**. **Because of the directionality of the RNN, for every time-step, it will predict the next point given only the preceding ones.**

In this PC-lab, we will use the hour of the timepoint as covariate.

For training, we can't put all days in to our model as one sample. Just like the reason for doing batches in other networks is that: it is more computationally efficient, and it allows us to have training steps on different parts of data with some stochasticity to it, allowing us to jump out of local minima.

**For RNN, another reason is that our "actual" neural network depth is essentially decided by our input length**, so if we send in a sample containing a thousand input tokens, we also backpropagate through a thousand layers, and our computers will surely crash. In addition, it is not reasonable to assume the number of cyclists hundreds of days past still influences the number of cyclists now. So, the problem of batching our sequence becomes one of weighing two factors: how long of a sequence can our model handle, and how much context (in number of preceding tokens) do our models need for prediction?

Here we will take a batch size of 48 as a default, meaning that our samples will always coincide with two consecutive days.

In [ ]:
def generate_batches(sequence, seqlen = 48):
    batches = []
    for i in np.arange(0, len(sequence) - seqlen, seqlen):
        batches.append(sequence[i:i+seqlen])
    return torch.stack(batches)

train_batches = generate_batches(torch.tensor(train_data[["Hour", "Totaal"]].values.astype(np.float32)))

Let's see how a sample looks like:

In [ ]:
train_batches.shape

In [ ]:
train_batches[0]

We could give our input to the model like this, as the hour is a numerical value which can be interpreted using linear layers. It would make more sense to treat the hour variable as categorical and encode it using dummy variables (one-hot encoding).

In [ ]:
one_hot_hour = torch.nn.functional.one_hot(train_batches[:, :, 0].long())

one_hot_hour.shape

In [ ]:
train_batches = torch.cat([train_batches[:, :, [1]], one_hot_hour], axis = 2)

In [ ]:
train_batches.shape

In [ ]:
train_batches[0]

Let's min-max scale the outputs. Remember: if you don't do this, your loss function (e.g. MSELoss) will be of a very big scale, affecting learning (i.e. you will need lower learning rates).
Hence, scaling allows us to make a better guess as to what a good learning rate will be.

In [ ]:
max_cyclists_train = train_batches[:, :, 0].max()
train_batches[:, :, 0] = train_batches[:, :, 0] / max_cyclists_train
train_batches[0]

Finally, let's prepare some batches in a similar way for the test set:

In [ ]:
test_batches = generate_batches(torch.tensor(test_data[["Hour", "Totaal"]].values.astype(np.float32)))
one_hot_hour = torch.nn.functional.one_hot(test_batches[:, :, 0].long())
test_batches = torch.cat([test_batches[:, :, [1]], one_hot_hour], axis = 2)
test_batches[:, :, 0] = test_batches[:, :, 0] / max_cyclists_train
test_batches[0]

### RNNs in PyTorch

In this PC-lab we will use the GRU, but note that other RNN-types work similarly in PyTorch.

[Documentation for the GRU](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html)

Extra note: The weird shape expectations (such as not expecting batches to come first by default) are a consequence of optimizations that PyTorch has implemented so the RNNs run efficiently on data with variable input sequence lengths (such as sentences). For this PC lab, we have batched our sequences so that they have constant sequence length, so we can add the argument `batch_first = True`.

Let's create a GRU and some toy data to see how it all works:

In [ ]:
gru = nn.GRU(input_size = 64, hidden_size = 512, batch_first = True)

In [ ]:
x = torch.randn(2, 50, 64)
output, h_n = gru(x)
output.shape, h_n.shape

Explanation of the outputs: `h_n` is the hidden representation of the last hidden memory cell. It can be seen as a summarized representation of the content of the whole input (if one wants a single output for a whole sequence as in e.g. sentence classification). `output` will return the output representation of the RNN for every input token. (Look back at the picture in the introduction of this part of the PC lab (Section 2.1) for more intuition as to when to use what outputs of the RNN)

For autoregressive modeling of time series, we should have an output for every input, mainly: the prediction of the next timepoint. For our purpose, we are interested to predict a single output per timepoint, whereas we can see that this is not the case for our GRU model as it is now. We can remedy this with a simple linear layer

<div class="alert alert-success">

<b>EXERCISE:</b>
<p> Implement an autoregressive GRU for cycler forecasting by completing the code below. The model should contain a recurrent layer, and a layer that takes the outputs of the GRU at each timestep and manipulates their dimensions so that the output dimensionality of each token is equal to one. Keep in mind how many input variables we have in our dataset. (Look at the $batches$ variable).
</p>

</div>

Test your model with some toy data down below.



In [ ]:
class CyclerForecaster(nn.Module):
    # def __init__() ....

    # def forward() ....

In [ ]:
model = CyclerForecaster()

x = torch.randn(2, 48, 25)

y = model(x)

y.shape

To create an input and an output, we have to do the above-mentioned shifting. In practice, this means that we take all but the last timepoints to create X, and all but the first timepoints to create Y, hence creating shifted X,y pairs.

Additionally, for y, we only want to keep the first variable, meaning the number of cyclists itself.

In [ ]:
X_train = train_batches[:, :-1]

y_train = train_batches[:, 1:]
y_train = y_train[:, :, [0]]

print(X_train.shape, y_train.shape)

Doing the same for the test data, and putting the things into datasets and dataloaders.

In [ ]:
X_test = test_batches[:, :-1]
y_test = test_batches[:, 1:, [0]]

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 8, shuffle = True)

test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 8, shuffle = True)

Note that we are just taking the first 80% of the data as training and the last 20% as validation set. Normally, we shuffle our data so that we are not biased. In this case, however, we can make a case for doing it our way: because the samples are ordered by day and month, the last samples will be from the summer months, where we may expect different patterns (June: exams, July: vacations).

<div class="alert alert-success">

<b>EXERCISE:</b>
<p> Implement the training loop for the Cycler Forecaster using the same principles from last PC labs. Keep in mind that unlike previous PC labs, we are now training a regression task. You will also need to up the number of epochs as our dataset is quite small and each epoch only constitutes a small number of training steps.
</p>

</div>


In [ ]:
# YOUR CODE HERE

To evaluate our model beyond looking at a loss function going down: we can look at the autoregressive results for a random test sample:

In [ ]:
import matplotlib.pyplot as plt

plt.plot(np.arange(len(y_test[0])), y_test[0])
plt.plot(np.arange(len(y_test[0])), model(X_test[[0]]).detach().squeeze(0).numpy())
plt.legend(["True", "Predicted"])
plt.show()

You should see that at the beginning of the second day, the model overshoots its prediction by a wide margin. This is because most probably the next day is a weekend day. After that, for the prediction of the next step, we give it the true input of that (previously-wrongly predicted) timestep, and because it recognizes that by this low value it should be a weekend day, the subsequent predictions are also low.



This is not really forecasting though: at every timestep, we are predicting only one timestep (hour) in advance. To really do forecasting for a longer time limit, we should feed the predictions of the model back into the model, like this:

<img src="https://raw.githubusercontent.com/gdewael/teaching/main/predmod/RNNs/generation.png" width = 500>

To perform this, we will create a helper function that extracts the next step:

In [ ]:
def generate_next_timestep(previous, model):
    with torch.no_grad():
        output = model(previous.unsqueeze(0))[0, -1]

    return output

def generate_n_timesteps(previous, model, n = 5):
    for _ in range(n):
        prediction = generate_next_timestep(previous, model)
        # to make a next prediction, we not only need to add the prediction but also the covariates (hour) to our next input:
        next_timestep_hour = nn.functional.one_hot((previous[-1, 1:].argmax() + 1) % 24, num_classes = 24)
        new_input = torch.cat([prediction, next_timestep_hour])
        previous = torch.cat([previous, new_input.unsqueeze(0)])

    return previous[:, 0]

In [ ]:
sample_index = 0 # any index in the test set
priming_points = ... # how many observed timepoints to give the model to start predicting from
generate_steps = ... # how many timepoints in the future the model should forecast


plt.plot(np.arange(len(X_test[sample_index])), X_test[sample_index, :, 0])
predictions = generate_n_timesteps(X_test[sample_index, :priming_points], model, n = generate_steps).numpy()
plt.plot(np.arange(len(predictions)), predictions)
plt.axvline(x = priming_points-1, color = "green", linestyle = ":")
plt.legend(["True", "Predicted"])
plt.show()

<div class="alert alert-success">

<b>THOUGHT EXERCISE:</b>
<p> Play around with the code for forecasting bicyclists by changing up which sample you forecast ("sample_index"), how much initial input points you give the model ("priming points") and how far in time it should generate after that ("generate steps"). Can you find obvious failure states? Can you think of ways how to remedy these failure modes?
</p>

</div>

## Partim 2: time series classification

In the next part, we will explore a many-to-one scenario. To keep things simple, we will work on the same dataset and try to predict from the series of a single day whether that day was a weekend or a weekday.

To do this, we first need to know which days were weekdays:

In [ ]:
train_data

Following code adds an indicator variable to the data that signifies if the date is a weekday or not. We'll use this as "y" values to predict

In [ ]:
from datetime import datetime
def isweekday(date):
    return datetime.fromisoformat(date).weekday() < 5

train_data["Date"] = train_data["Date_hour"].str.split("T", expand = True)[0]
test_data["Date"] = test_data["Date_hour"].str.split("T", expand = True)[0]

train_data["isweekday"] = [int(isweekday(i)) for i in train_data["Date"]]
test_data["isweekday"] = [int(isweekday(i)) for i in test_data["Date"]]
train_data



With this data added, we can process our data into X and y:

In [ ]:
train_batches = generate_batches(torch.tensor(train_data[["isweekday", "Hour", "Totaal"]].values.astype(np.float32)), seqlen = 24)
one_hot_hour = torch.nn.functional.one_hot(train_batches[:, :, 1].long())
train_batches = torch.cat([train_batches[:, :, [0, 2]], one_hot_hour], axis = 2)
train_batches[:, :, 1] = train_batches[:, :, 1] / max_cyclists_train

y_train = train_batches[:, 0, 0]
X_train = train_batches[:, :, 1:]
X_train.shape, y_train.shape

In [ ]:
X_train[0]

The same for test data:

In [ ]:
test_batches = generate_batches(torch.tensor(test_data[["isweekday", "Hour", "Totaal"]].values.astype(np.float32)), seqlen = 24)
one_hot_hour = torch.nn.functional.one_hot(test_batches[:, :, 1].long())
test_batches = torch.cat([test_batches[:, :, [0, 2]], one_hot_hour], axis = 2)
test_batches[:, :, 1] = test_batches[:, :, 1] / max_cyclists_train

y_test = test_batches[:, 0, 0]
X_test = test_batches[:, :, 1:]
X_test.shape, y_test.shape

Examine if the shapes are conforming to your expectations as to what we expect it to be for timeseries classification. What does every dimension in X and y signify?

In [ ]:
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 8, shuffle = True)

test_dataset = torch.utils.data.TensorDataset(X_test, y_test)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 8, shuffle = True)

<div class="alert alert-success">

<b>EXERCISE:</b>
<p> Implement a time series classification model by copying the code above and adapting it.

Steps you need to take:
- Change the RNN model so that it is adapted for many to one tasks
- Change the loss function for binary classification
- Implement a way to keep track of accuracies instead of only losses
</p>

</div>

You should be able to obtain an accuracy of at least 85%.

In [ ]:
class TimeseriesClassifier(nn.Module):
    # def __init__() ....

    # def forward() ....

In [ ]:
# TRAINING CODE GOES HERE ...

Let's plot all the test data and see what the model still got wrong. In the following code. Green timeseries are weekdays and blue timeseries are weekends. The transparent ones are correctly predicted, whereas the dotted ones are predicted wrongly. (So a blue dotted line means a weekend predicted as being a weekday and vice-versa).

In [ ]:
preds = (model(X_test) > 0).detach().int().numpy()
trues = y_test.numpy()

for i in range(len(X_test)):
    plt.plot(np.arange(24), X_test[i, :, 0].numpy(),
             color = ["blue", "green"][int(trues[i])],
             ls = [":", "-"][int(trues[i] == preds[i])],
             alpha = [1, 0.1][int(trues[i] == preds[i])],
            )